In [ ]:
# RNN Network Demp

In [18]:
# Simple Text Generating RNN using PyTorch (Word-Level)

# Import required libraries
import torch
import torch.nn as nn
import numpy as np
import re

# Function to read text from a file
def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Read text data from a file
file_path = 'data/grimm.txt'  # Replace with your text file path
text = read_text(file_path)

# Preprocess text: Convert to lowercase and split into words
text = text.lower()
words = re.findall(r'\b\w+\b', text)

# Add a special token for unknown words
words_set = sorted(set(words))
words_set.append('<unk>')  # Add unknown token to the set
word_to_index = {word: i for i, word in enumerate(words_set)}
index_to_word = {i: word for i, word in enumerate(words_set)}

# Prepare the dataset
maxlen = 10  # Length of extracted word sequences
step = 1     # Step size for creating sequences
sentences = []
next_words = []

for i in range(0, len(words) - maxlen, step):
    sentences.append(words[i: i + maxlen])
    next_words.append(words[i + maxlen])

# Convert data to PyTorch tensors
X = np.zeros((len(sentences), maxlen), dtype=np.int64)  # Changed to int64 for indices
y = np.zeros((len(sentences), len(words_set)), dtype=np.float32)  # Remain float for the target

for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        X[i, t] = word_to_index[word]
    y[i, word_to_index[next_words[i]]] = 1

X = torch.tensor(X, dtype=torch.long)  # Ensure the input tensor is of type Long
y = torch.tensor(y, dtype=torch.float32)  # Ensure the target tensor is of type Float

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        # Embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)
        # Simple RNN layer
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # Pass input through embedding layer
        x = self.embedding(x)
        # Pass embedding through RNN
        out, hidden = self.rnn(x, hidden)
        # Pass RNN output through fully connected layer
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros
        return torch.zeros(1, batch_size, self.hidden_size)

# Set parameters
input_size = len(words_set)
hidden_size = 128
output_size = len(words_set)
num_epochs = 50
learning_rate = 0.01

# Initialize the model, loss function, and optimizer
model = RNNModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    hidden = model.init_hidden(X.size(0))
    outputs, hidden = model(X, hidden)
    loss = criterion(outputs, torch.argmax(y, dim=1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 2 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

# Function to generate text
def generate_text(model, seed_text, length):
    model.eval()
    generated = seed_text
    hidden = model.init_hidden(1)
    
    seed_words = seed_text.lower().split()
    for _ in range(length):
        x_pred = torch.zeros(1, maxlen, dtype=torch.long)
        for t, word in enumerate(seed_words[-maxlen:]):
            if word in word_to_index:
                x_pred[0, t] = word_to_index[word]
            else:
                x_pred[0, t] = word_to_index['<unk>']  # Use <unk> if the word is not in the index
        
        with torch.no_grad():
            output, hidden = model(x_pred, hidden)
            next_index = torch.argmax(output).item()
            next_word = index_to_word[next_index]
            
        generated += ' ' + next_word
        seed_words.append(next_word)

    return generated

# Generate text using the trained model
seed_text = "kinder und "
generated_text = generate_text(model, seed_text, 50)
print(f"Generated text:\n{generated_text}")



Epoch [0/50], Loss: 8.9145
Epoch [2/50], Loss: 7.8911
Epoch [4/50], Loss: 6.6944
Epoch [6/50], Loss: 6.5158
Epoch [8/50], Loss: 6.2867
Epoch [10/50], Loss: 6.1092
Epoch [12/50], Loss: 5.9359
Epoch [14/50], Loss: 5.7665
Epoch [16/50], Loss: 5.6013
Epoch [18/50], Loss: 5.4321
Epoch [20/50], Loss: 5.2668
Epoch [22/50], Loss: 5.1081
Epoch [24/50], Loss: 4.9528
Epoch [26/50], Loss: 4.8053
Epoch [28/50], Loss: 4.6668
Epoch [30/50], Loss: 4.5343
Epoch [32/50], Loss: 4.4090
Epoch [34/50], Loss: 4.2908
Epoch [36/50], Loss: 4.1783
Epoch [38/50], Loss: 4.0708
Epoch [40/50], Loss: 3.9682
Epoch [42/50], Loss: 3.8692
Epoch [44/50], Loss: 3.7734
Epoch [46/50], Loss: 3.6803
Epoch [48/50], Loss: 3.5898
Generated text:
kinder und  der der der der der der der der wirth kam ein und als es sich auf den weg und sprach der junge könig und sprach er zu dem backofen gelangte und sprach der junge könig und sprach er zu dem backofen gelangte und sprach der junge könig und sprach er
